# Running Python, C or R Code from Julia

This notebook first provides a basic introduction to how to run Python, using the `PythonCall.jl` package and also C code (requires no package) from Julia. Please see the [PythonCall.jl](https://github.com/JuliaPy/PythonCall.jl) homepage for instructions for how to either use an existing Python installation, or let the package make one.
 
An alternative package (not used here) for running Python is [PyCall.jl](https://github.com/JuliaPy/PyCall.jl), which is a slightly older approach.

The second part of the notebook runs R code, using the [RCall.jl](https://github.com/JuliaInterop/RCall.jl) package, from Julia. See the the RCall.jl homepage (the installation instructions) for how to use your existing R installation or for letting RCall.jl install R for you.

In [1]:
using Printf, DelimitedFiles
include("src/printmat.jl");

# Load Data

In [2]:
x = readdlm("Data/MyData.csv",',',skipstart=1)  #reading the csv file

(Rme,Rf,R) = (x[:,2],x[:,3],x[:,4])  #creating variables from columns of x
y  = R - Rf                          #do R .- Rf if R has several columns

c = ones(length(Rme))
x = [c Rme]

b = x\y
println("OLS coeffs according to Julia")
printmat(b)

OLS coeffs according to Julia
    -0.504
     1.341



# Python

You need Python's `statsmodels` package installed to run the code below. If you have let PythonCall install Python for you, use [CondaPkg.jl](https://github.com/JuliaPy/CondaPkg.jl) to add packages: `import CondaPkg; CondaPkg.add("statsmodels")`.

In the next cells we *(a)* load the PythonCall.jl package and activates the (Python) package `statsmodels`; *(b)* call some functions (eg. `OLS()`) from statsmodels.

In many cases, you can use Julia arrays (like `y` and `x` in the cells below) in a call on a Python function. In other cases (like here), you need to convert them to numpy arrays.

In [3]:
using PythonCall
sm = pyimport("statsmodels.api")     #activate this package and call it `sm`
np = pyimport("numpy")
println("")

In [4]:
resultsP = sm.OLS(np.array(y), np.array(x)).fit()   #np.array() to convert to Python
println(resultsP.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.519
Model:                            OLS   Adj. R-squared:                  0.518
Method:                 Least Squares   F-statistic:                     416.2
Date:                Mon, 17 Nov 2025   Prob (F-statistic):           2.72e-63
Time:                        11:44:09   Log-Likelihood:                -1241.7
No. Observations:                 388   AIC:                             2487.
Df Residuals:                     386   BIC:                             2495.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.5042      0.305     -1.654      0.0

In [5]:
#foreach(println,pydir(resultsP))          #run this to see all attributes

In [6]:
b_P1 = PyArray(resultsP.params)                   #a Julia view of the Python array,
b_P2 = pyconvert(Array{Float64}, resultsP.params)   #or copy/convert to a Julia array

printblue("Comparing the estimates in Julia and Python:")
printmat(b,b_P1,b_P2;colNames=["Julia","Julia view","Py->Ju"],width=15)

Comparing the estimates in Julia and Python:
          Julia     Julia view         Py->Ju
         -0.504         -0.504         -0.504
          1.341          1.341          1.341



We can also run Python code (quoted in """ """) directly as in the next cell.

In [7]:
@pyexec (x=x, y=y) => """
import numpy as np
x = np.array(x)
y = np.array(y)
xx = np.matmul(x.T, x)
xy = np.matmul(x.T, y)
b_p = np.linalg.solve(xx, xy)
""" => b_p

b_P3 = PyArray(b_p)
printmat(b_P3)

    -0.504
     1.341



# C

This section shows some simple examples of how to call a C function. The functions are in the file `My_C_Stuff.c` (printed in the next cell). The first function `c_dot` defines a dot product between two vectors and the second function `c_ols` a simple linear regression.

In [8]:
#println(read("Data/My_C_Stuff.c",String))   #uncomment to see the c file

To compile to a dynamlic library (dll on windows), I use gcc (for x86_64) from [mingw-64](http://mingw-w64.org)
and run the following in the mingw terminal
```
gcc -shared -fPIC My_C_Stuff.c -o My_C_Stuff.dll
```

To call the C functions, place the dll file in the current folder and then run the following cells.

## A Function which Returns a Number

In the next example, we use the function `c_dot` in `My_C_Stuff.dll`. The function calculates the inner product of two vectors.

The details are:
1. `mylibc.c_dot` is the library.function
2. `length(y)::Csize_t` is the first input and its type (an integer indicating the number of elements in `y`)
3. `y::Ptr{Float64}` is the second input (a pointer to an array of Floats) and similarly for `x2`
4. `Float64` is the type of the output

(We could potentially wrap this in a Julia function that checks for the right input types and outputs the result.)

In [9]:
mylibc = "My_C_Stuff.dll"

x2     = x[:,2];               #get a vector with the regressor values

z = @ccall mylibc.c_dot(length(y)::Csize_t, y::Ptr{Float64}, x2::Ptr{Float64})::Float64

printlnPs("The inner product of x2 and y in Julia and C:  ",x2'y," ",z)

The inner product of x2 and y in Julia and C:   11071.648           11071.648


## A Function which Returns a Vector

The details are as above, except that 
1. `mylibc.c_ols` is the library.function
2. `Cvoid` is the type of the output, which here indicates that the function does not have an output. Rather, the function modifies the vector `b_c` by putting the OLS results there.

In [10]:
b_c = zeros(2)          #where C will store the regression results

@ccall mylibc.c_ols(length(y)::Csize_t, y::Ptr{Float64}, x2::Ptr{Float64}, b_c::Ptr{Float64})::Cvoid

println("Comparing the estimates in Julia and C")
printmat([b b_c];colNames=["Julia","C"])

Comparing the estimates in Julia and C
     Julia         C
    -0.504    -0.504
     1.341     1.341



# R

To avoid naming conflicts with the Python packges, *restart the kernel* and work from here.

The first cell loads the data into Julia (again). The subsequent cells load the RCall.jl package, transfers the data from Julia to R (`@rput`) and then run an OLS regression (`reval()`).

In [1]:
using Printf, DelimitedFiles
include("src/printmat.jl")          #quickly redoing the Julia part

x = readdlm("Data/MyData.csv",',',skipstart=1)  #reading the csv file
(Rme,Rf,R) = (x[:,2],x[:,3],x[:,4])  #creating variables from columns of x
y  = R - Rf                          #do R .- Rf if R has several columns
c = ones(length(Rme))
x = [c Rme]
b = x\y
println("OLS coeffs according to Julia")
printmat(b)

OLS coeffs according to Julia
    -0.504
     1.341



In [2]:
using RCall

In [3]:
@rput x y                          #send x and y to R

resultsR = reval("summary(mod <- lm(y ~ x-1))")  #run R code
println(resultsR)                                #and print output

RObject{VecSxp}

Call:
lm(formula = y ~ x - 1)

Residuals:
    Min      1Q  Median      3Q     Max 
-17.981  -3.131  -0.359   2.281  52.361 

Coefficients:
   Estimate Std. Error t value Pr(>|t|)    
x1 -0.50416    0.30483  -1.654    0.099 .  
x2  1.34105    0.06573  20.401   <2e-16 ***
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1

Residual standard error: 5.954 on 386 degrees of freedom
Multiple R-squared:  0.5194,	Adjusted R-squared:  0.5169 
F-statistic: 208.6 on 2 and 386 DF,  p-value: < 2.2e-16




In [4]:
println(names(resultsR))           #print all keys (field names)

[:call, :terms, :residuals, :coefficients, :aliased, :sigma, :df, Symbol("r.squared"), Symbol("adj.r.squared"), :fstatistic, Symbol("cov.unscaled")]


In [5]:
b_R = rcopy(resultsR[:coefficients])  #the numerical results are now a Julia array

printblue("Comparing the estimates in Julia and R")
printmat([b b_R[:,1]];colNames=["Julia","R"])

Comparing the estimates in Julia and R
     Julia         R
    -0.504    -0.504
     1.341     1.341



In [6]:
R"model <- lm(y ~ x-1)"                              #an alternative approach
resultsR2 = rcall(:summary, R"model")
b_R2      = rcopy(resultsR2[:coefficients])
printmat([b b_R[:,1] b_R2[:,1]];colNames=["Julia","R","R ver, 2"])

     Julia         R  R ver, 2
    -0.504    -0.504    -0.504
     1.341     1.341     1.341



We can also run blocks of R code `R""" code """` like in the cell below. 
Do `@rput x y` or use `$x` and `$y` in the code below.

In [7]:
#solve x'x*b = x'y
R"""
xx <- t(x)%*%x
xy <- t(x)%*%y
b_R <- solve(xx,xy)
"""

RObject{RealSxp}
           [,1]
[1,] -0.5041626
[2,]  1.3410486
